In [1]:
import numpy as np 


In [7]:
N = np.zeros((7,2))
Q = np.zeros((7,2))
states = [2,1,1,0]
rewards = [0,0,0,1]
actions =   [1,1,1,1]
G = 0
for t in range(0,len(states))[::-1]:
    G = .9 * G + rewards[t]
                
    N[states[t],int(actions[t])] += 1
    Q[states[t],int(actions[t])] += (G-Q[states[t],int(actions[t])])/N[states[t],int(actions[t])]
print(Q)

[[0.    1.   ]
 [0.    0.855]
 [0.    0.729]
 [0.    0.   ]
 [0.    0.   ]
 [0.    0.   ]
 [0.    0.   ]]


In [8]:
np.random.random()

0.40540776318336125

In [13]:
np.unique([1,2,3,3])

array([1, 2, 3])

In [16]:
np.unique([[1,1],[1,1],[3,1],[1,3]], return_counts=True, axis=0)

(array([[1, 1],
        [1, 3],
        [3, 1]]),
 array([2, 1, 1]))

In [53]:
a = np.zeros((3,4)).astype(int)

In [54]:
a[[[0,0],[0,1]]] = 1

<ipython-input-54-8af4f4519b45>:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  a[[[0,0],[0,1]]] = 1


In [55]:
a

array([[1, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [56]:
a[np.sum(a, axis=-1)==0]=1

In [57]:
a

array([[1, 1, 0, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1]])

In [60]:
(a.T * [1,0,1]).T

array([[1, 1, 0, 0],
       [0, 0, 0, 0],
       [1, 1, 1, 1]])